In [1]:
import pandas as pd
from datetime import timedelta


class Preprocessor : 

    def __init__ (self , data_path) : 
        self.df=pd.read_csv(data_path, low_memory=False)
        self.thresholds = {
            'MV1': (4.342303276, 488.2485352),
            'MV2': (4.300886154, 1237.251953),
            'MV3': (2.820113897, 562.4943237),
            'MV4': (-1.878349543, 258.3875122),
            'MV5': (18.381464, 757.949707),
            'MV6': (0, 300),
            'MV7': (2.225207329, 39.08103943),
            'MV8': (0.732448936, 39.08103943),
            'MV9': (1.5459584, 39.08103943),
            'MV10': (4.465659618, 106.5072937),
            'MV11': (2.27641654, 39.08103943),
            'MV12': (5.236536503, 114.5099945),
            'CV1': (25.10530472, 472.3631897)
        }
    
        # Convert the "Date"column to datetime
        columns_to_convert = self.df.columns.difference(['Date'])  # Exclude the "Date" column
        self.df[columns_to_convert] = self.df[columns_to_convert].apply(pd.to_numeric, errors='coerce')

        self.df['Date'] = pd.to_datetime(self.df['Date'], format='%m/%d/%Y %H:%M')
        self.last_recorded_date = self.df['Date'].max()  # Get the maximum (most recent) date


    def filter_last_24h(self):    
        # Calculate the date and time 24 hours ago from the last recorded date
        twenty_four_hours_ago = self.last_recorded_date - timedelta(hours=24)

        # Filter the dataset for rows recorded in the last 24 hours
        self.filtred_24h =  self.df[(self.df['Date'] >= twenty_four_hours_ago) & (self.df['Date'] <= self.last_recorded_date)]
        return self.filtred_24h

    def CV_sammary(self, df):
        column = 'CV1'
        data = df[[column]].copy()
        total_rows = len(data)
        thr = 5
        # Mark outliers
        outliers = (data < self.thresholds[column][0]) | (data > self.thresholds[column][1])
        # Calculate the percentage of outliers for this variable
        percentage_outliers = (outliers.sum() / len(data)) * 100

        # Count NaN and missing values
        missing_count = data[column].isnull().sum()
        percentage_missing_count = (missing_count.sum() / len(data)) * 100

        # Calculate the frequency of each unique value
        unique_values = data[column].value_counts()
            # Calculate the percentage of frozen values for this variable
        frozen_count = sum(unique_values[unique_values <= thr])
        percentage_frozen = (frozen_count / total_rows) * 100

        return percentage_outliers[column], percentage_missing_count, percentage_frozen

    def summary(self, df):
        data = df.copy()
        # Create a dictionary to store the percentage of outliers, NaN, and missing values for each variable
        summary_dict = {'Variable': [], 'Outlier': [],  'Missing': [], 'Frozen': []}
        # Need to add, % frozen values
        total_rows = len(data)
        thr = 5
        # Loop through acceptance_ranges to mark outliers and calculate counts of NaN and missing values for each variable
        for column, (min_value, max_value) in self.thresholds.items():
            # Mark outliers
            outliers = (data[column] < min_value) | (data[column] > max_value)
            data[column + '_outlier_'] = outliers

            # Calculate the percentage of outliers for this variable
            percentage_outliers = (outliers.sum() / len(data)) * 100

            # Count NaN and missing values
            missing_count = data[column].isnull().sum()
            percentage_missing_count = (missing_count.sum() / len(data)) * 100

            # Calculate the frequency of each unique value
            unique_values = data[column].value_counts()
                # Calculate the percentage of frozen values for this variable
            frozen_count = sum(unique_values[unique_values <= thr])
            percentage_frozen = (frozen_count / total_rows) * 100

            # Append values to the summary dictionary
            summary_dict['Variable'].append(column)
            summary_dict['Outlier'].append(percentage_outliers)
            #summary_dict['percentage NaN Count (%)'].append(percentage_nan_count)
            summary_dict['Missing'].append(percentage_missing_count )
            summary_dict['Frozen'].append(percentage_frozen)

        # Create a DataFrame from the summary_dict
        summary_df = pd.DataFrame(summary_dict)
        return summary_df
        



In [2]:
# Load and preprocess data
preprocessor = Preprocessor("/Users/ghita/Desktop/fm/projects/AGRO-Dashboard/P075-6 Months_MVCV_18June23.csv")

In [3]:
# Filter Last 24 hours
df = preprocessor.filter_last_24h()
df

,Date,MV1,MV2,MV3,MV4,MV5,MV6,MV7,MV8,MV9,MV10,MV11,MV12,CV1
260640,2023-06-11 16:10:00,372.923981,368.989502,346.101715,21.702271,752.819885,-0.406918,39.067482,39.045498,39.067482,98.003250,39.067482,103.658821,369.172394
260641,2023-06-11 16:11:00,373.020508,366.568909,342.644501,21.702271,752.840637,-0.406918,39.067482,39.045746,39.067482,98.021683,39.067482,103.637787,369.268372
260642,2023-06-11 16:12:00,373.117004,364.366974,339.393646,21.702271,752.861328,-0.406918,39.067482,39.045994,39.067482,98.040123,39.067482,103.616753,369.364349
260643,2023-06-11 16:13:00,373.213531,364.910248,339.856903,21.702271,752.882080,-0.406918,39.067482,39.046242,39.067482,98.058556,39.067482,103.595726,369.460327
260644,2023-06-11 16:14:00,373.310059,365.453491,340.320190,21.702271,752.902832,-0.406918,39.067482,39.046494,39.067482,98.076996,39.067482,103.574692,369.556305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262076,2023-06-12 16:06:00,373.143433,353.999451,344.168549,22.787384,750.084717,-0.271278,39.081039,39.081039,39.081039,106.494888,39.081039,114.474098,348.438538
262077,2023-06-12 16:07:00,373.143433,354.018310,344.116638,22.787384,750.084717,-0.271278,39.081039,39.081039,39.081039,106.507294,39.081039,114.509995,348.457092
262078,2023-06-12 16:08:00,373.143433,354.018310,344.116638,22.787384,750.084717,-0.271278,39.081039,39.081039,39.081039,106.507294,39.081039,114.509995,348.457092
262079,2023-06-12 16:09:00,373.143433,354.018310,344.116638,22.787384,750.084717,-0.271278,39.081039,39.081039,39.081039,106.507294,39.081039,114.509995,348.457092


In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import joblib
import time
from sklearn.metrics import r2_score
from IPython.display import clear_output


class Predictor :
    def __init__(self, data,
                model_path = '/Users/ghita/Desktop/fm/projects/AGRO-Dashboard/ml_tools/P075_24h_3h_CNNLSTM.h5',
                scaler1_path = '/Users/ghita/Desktop/fm/projects/AGRO-Dashboard/ml_tools/scaler1.pkl',
                scaler2_path = '/Users/ghita/Desktop/fm/projects/AGRO-Dashboard/ml_tools/scaler2.pkl'):
       
        # Charger le modèle pré-entraîné 
        self.model = load_model(model_path)
        
        # Charger le premier scaler depuis le fichier
        with open(scaler1_path, 'rb') as file1:
            self.sc1 = joblib.load(file1)

        # Charger le deuxième scaler depuis le fichier
        with open(scaler2_path, 'rb') as file2:
            self.sc2 = joblib.load(file2)
        
        # Loading and preprocessing data
        #data["Date"] = pd.to_datetime(data["Date"])
        #data = data.sort_values(by='Date', ascending=True)
        #data.set_index('Date', inplace=True)

        # Convert all object types to numeric types
        #data = data.apply(pd.to_numeric, errors='coerce')

        # Removing outliers
        #Q1 = data.quantile(0.25)
        #Q3 = data.quantile(0.75)
        #IQR = Q3 - Q1
        #data = data[~((data < (Q1 - 1.5 * IQR)) | (data > (Q3 + 1.5 * IQR))).any(axis=1)]

        # Using interpolation to fill missing values in the data
        data = data.ffill()
        data = data.dropna()

        label = ['CV1']
        features = ['MV1', 'CV1']
        test_size = 0.01
        input_seq = 1440  # 24 hours
        output_seq = 180  # 3 hours
        # Split data
        #_, self.X_test, _, self.y_test = train_test_split(data[features], data[label], test_size=test_size, shuffle=False)

        self.X_test = data[features]
        self.start_index = 0
        self.end_index = 16

        # Lire de nouvelles données du fichier de données
        #self.new_data = pd.read_csv('/Users/ghita/Desktop/fm/projects/AGRO-Dashboard/P104-6 Months_MVCV_18June23.csv', low_memory=True)  # Use the same data path

    def nRMSE(pred, Y_test):
        RF_test_error = np.sqrt(1/len(Y_test) * (sum((pred - Y_test)**2))) / (np.mean(Y_test)) * 100
        nrmse = [round(i, 2) for i in RF_test_error]
        return nrmse

    def nMAE(pred, Y_test):
        RF_test_error_NMAE = 100 * (sum(abs(pred - Y_test)) / sum(Y_test))
        nmae = [round(i, 2) for i in RF_test_error_NMAE]
        return nmae
    
    def predict (self, label = ['CV1'], features = ['MV1', 'CV1'], input_seq = 1440 , output_seq = 180):

        # Écraser les 15 lignes les plus anciennes de X_test
        #self.X_test = self.X_test.iloc[15:]

        # Sauvegarder l'état initial de X_test
        #X_test_initial = self.X_test.copy()

        # Sauvegarder l'état initial de y_test
        #y_test_initial = self.y_test.copy()

        # Ajouter les 15 prochaines lignes de nouvelles données à X_test
        #X_test_new = self.new_data[features].iloc[self.start_index:self.end_index]

        # Ajouter les données normalisées à X_test
        #self.X_test = pd.concat([self.X_test, pd.DataFrame(X_test_new, columns=features)])

                # Mettre à jour les données de test
        X_test_sc = self.sc1.transform(self.X_test)
        #y_test_sc = self.sc2.transform(self.y_test)


        # Split data en séquences
        no_record_test = X_test_sc.shape[0]
        self.X_test = []
        #self.y_test = []

        for i in range(input_seq, no_record_test - output_seq):
            self.X_test.append(X_test_sc[i - input_seq:i])
            #self.y_test.append(y_test_sc[i:i + output_seq])



        self.X_test = np.array(self.X_test)
        print(self.X_test.shape)
        #self.y_test = np.array(self.y_test)
        self.X_test = np.reshape(self.X_test, (self.X_test.shape[0], self.X_test.shape[1], self.X_test.shape[2]))

        # Faire des prédictions avec le modèle mis à jour
        preds = self.model.predict(self.X_test)
        #y_test_reshaped = self.y_test.reshape(-1, self.y_test.shape[-2])
        y_pred_reshaped = preds.reshape(-1, preds.shape[-2])
        #y_test_inverse = self.sc2.inverse_transform(y_test_reshaped)
        y_pred_inverse = self.sc2.inverse_transform(y_pred_reshaped)

                    # Evaluate the model (e.g., calculate RMSE)
        #rmse = np.sqrt(np.mean((y_pred_inverse  - y_test_inverse) ** 2))
        #r2 = r2_score(y_pred_inverse, y_test_inverse)

        # Tracer les prédictions
        #plt.plot(y_test_inverse[:,0], label='Real_values')
        #plt.plot(y_pred_inverse[:,0], label='Predicted_values')
        #plt.legend()

        # Restaurer l'état initial de X_test
        #self.X_test = X_test_initial.copy()
        # Restaurer l'état initial de X_test
        #self.y_test = y_test_initial.copy()

        # Mettre à jour les indices des prochaines données à ajouter
        self.start_index += 15
        self.end_index += 15

        return y_pred_inverse[:,0], #rmse, r2



In [11]:
features = ['MV1', 'CV1']
X_test = df[features][:1440]
X_test

,MV1,CV1
260640,372.923981,369.172394
260641,373.020508,369.268372
260642,373.117004,369.364349
260643,373.213531,369.460327
260644,373.310059,369.556305
...,...,...
262075,373.143433,348.383057
262076,373.143433,348.438538
262077,373.143433,348.457092
262078,373.143433,348.457092


In [12]:
model_path = '/Users/ghita/Desktop/fm/projects/AGRO-Dashboard/ml_tools/P075_24h_3h_CNNLSTM.h5'
scaler1_path = '/Users/ghita/Desktop/fm/projects/AGRO-Dashboard/ml_tools/scaler1.pkl'
scaler2_path = '/Users/ghita/Desktop/fm/projects/AGRO-Dashboard/ml_tools/scaler2.pkl'
       
# Charger le modèle pré-entraîné 
model = load_model(model_path)

# Charger le premier scaler depuis le fichier
with open(scaler1_path, 'rb') as file1:
    sc1 = joblib.load(file1)

# Charger le deuxième scaler depuis le fichier
with open(scaler2_path, 'rb') as file2:
    sc2 = joblib.load(file2)

/usr/local/lib/python3.9/site-packages/sklearn/base.py:288: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.2.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [23]:
X_test_sc = sc1.transform(X_test)
X_test_sc

array([[0.79300395, 0.56686947],
       [0.79364629, 0.56747853],
       [0.79428843, 0.56808759],
       ...,
       [0.7944643 , 0.43541282],
       [0.7944643 , 0.43541282],
       [0.7944643 , 0.43541282]])

In [25]:
X_test_sc = X_test_sc[np.newaxis, :, :]
X_test_sc.shape

(1, 1440, 2)

In [26]:
y = model.predict(X_test_sc)
y

1/1 [==============================] - 3s 3s/step


array([[[0.42423853],
        [0.4241089 ],
        [0.42402482],
        [0.42393425],
        [0.42380145],
        [0.42363203],
        [0.4236052 ],
        [0.42378137],
        [0.42391673],
        [0.42384914],
        [0.42380816],
        [0.4236601 ],
        [0.4233886 ],
        [0.42317757],
        [0.42297202],
        [0.42281425],
        [0.4227065 ],
        [0.42280692],
        [0.4230512 ],
        [0.42329878],
        [0.42349488],
        [0.42364854],
        [0.4237481 ],
        [0.42375064],
        [0.42361948],
        [0.42363173],
        [0.42369008],
        [0.4237868 ],
        [0.4237019 ],
        [0.42358768],
        [0.42346597],
        [0.42340356],
        [0.4232578 ],
        [0.42299503],
        [0.42292085],
        [0.42265305],
        [0.42246062],
        [0.42217088],
        [0.42196557],
        [0.42172408],
        [0.4217311 ],
        [0.421978  ],
        [0.4220635 ],
        [0.42188063],
        [0.42177323],
        [0

In [52]:
y_pred_reshaped = y.reshape(-1, y.shape[-2])
y_pred = sc2.inverse_transform(y_pred_reshaped)
y_pred

array([[346.69623, 346.67578, 346.66254, 346.6483 , 346.62735, 346.60065,
        346.59644, 346.62418, 346.6455 , 346.63486, 346.6284 , 346.60507,
        346.5623 , 346.52902, 346.49664, 346.47177, 346.45477, 346.4706 ,
        346.50912, 346.5481 , 346.57904, 346.60324, 346.61893, 346.61935,
        346.5987 , 346.6006 , 346.6098 , 346.62503, 346.61163, 346.59366,
        346.5745 , 346.56464, 346.5417 , 346.50024, 346.48856, 346.44638,
        346.41605, 346.3704 , 346.33804, 346.3    , 346.3011 , 346.34   ,
        346.35345, 346.32465, 346.30774, 346.3196 , 346.31006, 346.30176,
        346.29605, 346.29895, 346.33057, 346.34146, 346.36102, 346.38574,
        346.39545, 346.41168, 346.41684, 346.4113 , 346.3985 , 346.38467,
        346.38715, 346.4037 , 346.4069 , 346.37866, 346.34314, 346.29523,
        346.28375, 346.29187, 346.31384, 346.33017, 346.35266, 346.3795 ,
        346.41153, 346.4375 , 346.4539 , 346.4586 , 346.4394 , 346.43924,
        346.4427 , 346.43176, 346.4075

In [51]:
# Create a DatetimeIndex with a start time of '2023-06-12 16:10:00' and a frequency of one minute
date_rng = pd.date_range(start=preprocessor.last_recorded_date, periods=180, freq='1min')

# Create a DataFrame with the array and the DatetimeIndex
df_pred = pd.DataFrame(y_pred.T, index=date_rng)

# Rename the columns to 'date' and 'prediction'
df_pred = df_pred.reset_index()
df_pred.columns = ['date', 'prediction']

# Add one minute to each row in the 'date' column
#df_pred['date'] = df_pred['date'].apply(lambda x: x + pd.Timedelta(minutes=1))
df_pred


,date,prediction
0,2023-06-12 16:10:00,346.696228
1,2023-06-12 16:11:00,346.675781
2,2023-06-12 16:12:00,346.662537
3,2023-06-12 16:13:00,346.648285
4,2023-06-12 16:14:00,346.627350
...,...,...
175,2023-06-12 19:05:00,346.207611
176,2023-06-12 19:06:00,346.231384
177,2023-06-12 19:07:00,346.245148
178,2023-06-12 19:08:00,346.247345


In [67]:
def predict_3h(df, date, features):
    #date = df[['Date']].max()
    X_test = df[features][:1440]
    X_test_sc = sc1.transform(X_test)
    X_test_sc = X_test_sc[np.newaxis, :, :]
    y = model.predict(X_test_sc)
    y_pred_reshaped = y.reshape(-1, y.shape[-2])
    y_pred = sc2.inverse_transform(y_pred_reshaped)

    date_rng = pd.date_range(start=date, periods=180, freq='1min')
    # Create a DataFrame with the array and the DatetimeIndex
    df_pred = pd.DataFrame(y_pred.T, index=date_rng)

    # Rename the columns to 'date' and 'prediction'
    df_pred = df_pred.reset_index()
    df_pred.columns = ['date', 'prediction']

    return df_pred


In [72]:
predict_3h(preprocessor.filtred_24h, preprocessor.last_recorded_date, features = ['MV1', 'CV1'])

1/1 [==============================] - 0s 326ms/step


,date,prediction
0,2023-06-12 16:10:00,346.696228
1,2023-06-12 16:11:00,346.675781
2,2023-06-12 16:12:00,346.662537
3,2023-06-12 16:13:00,346.648285
4,2023-06-12 16:14:00,346.627350
...,...,...
175,2023-06-12 19:05:00,346.207611
176,2023-06-12 19:06:00,346.231384
177,2023-06-12 19:07:00,346.245148
178,2023-06-12 19:08:00,346.247345


In [80]:
import requests
import json
import pandas as pd

url = "https://eu-west-2.aws.data.mongodb-api.com/app/data-rpqya/endpoint/data/v1/action/find"

payload = json.dumps({
    "dataSource": "Cluster0",
    "database": "DB-P104",
    "collection": "SmartFarm",
    "sort": { "Date": -1 },
    "limit": 1
})
headers = {
    "apiKey": "nzyQOL6Q8MzrC7ReUgDVmiqVYcXh1o7SUdI4sEiVbjSx4TUrXz7tJVJlpV5lgPiV",
    "Content-Type": "application/ejson",
    "Accept": "application/json",
}

# Make the POST request
response = requests.request("POST", url, headers=headers, data=payload)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
else:
    print(f"Request failed with status code {response.status_code}")
# Extract the list of dictionaries
documents_list = data['documents']

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(documents_list)
df.drop("_id", axis=1, inplace= True)
df[['Date', 'MV1', 'MV2', 'MV3', 'MV4', 'MV5', 'MV6', 'MV7', 'MV8', 'MV9', 'MV10', 'MV11', 'MV12', 'CV1']]
df

,Date,MV12,CV1,MV9,MV8,MV10,MV14,MV4,MV1,MV13,MV3,MV2,MV5,MV11,MV6,MV7
0,2022-12-12T14:36:00Z,38.945404,-110.545944,38.931839,38.958969,38.931839,38.986092,-110.61245,-110.410309,38.93821,-110.563934,-110.453834,-110.666832,38.931839,-110.545944,-110.410309


In [76]:
data

{'document': {'Date': '2022-12-12T14:36:00Z',
  'MV12': 38.94540405,
  'CV1': -110.5459442,
  'MV9': 38.93183899,
  'MV8': 38.95896912,
  'MV10': 38.93183899,
  'MV14': 38.98609161,
  'MV4': -110.6124496,
  'MV1': -110.4103088,
  'MV13': 38.93820953,
  'MV3': -110.5639343,
  '_id': '6515f56448d45961b62c42f2',
  'MV2': -110.4538345,
  'MV5': -110.666832,
  'MV11': 38.93183899,
  'MV6': -110.5459442,
  'MV7': -110.4103088}}